# COMMANDS and IMPORTS

In [ ]:
import os
os.environ['BIN_DIR'] = "/bsuhome/zayahcortright/q-e/bin"
os.environ['ESPRESSO_PSEUDO'] = "/bsuhome/zayahcortright/q-e/pseudo"
os.environ['OMP_NUM_THREADS'] = "1"
os.environ['ASE_ESPRESSO_COMMAND'] = "module restore qe && srun --mpi=pmix -p short -N 2 -n 96 -t 24:00:00 --signal=USR1@300 $BIN_DIR/pw.x -nk 2 -in PREFIX.pwi > PREFIX.pwo"

In [1]:
from ase.build import surface
from ase.io import write
from ase.visualize import view
from ase.build import make_supercell
from ase.io import read
from ase.build import make_supercell, surface
from ase.calculators.espresso import Espresso
from ase.atom import Atom
from ase.build import add_adsorbate
import pandas as pd
import csv
from ase import io
import os



In [2]:
# Setup (General)
# Define material structure with either .xyz or .cif file
material = "SrTiO3_A.cif"

# Define the name of the save files
save_name = "SrTiO3_A"

# Directory to save the files internally (within the current folder)
save_dir = "SrTiO3_A_files"

# Define aborbate name
options = ["O", "H", "OOH", "OH", "H2O"]
abs_name = options[2]

# Pseudopotentials
psuedopotentials = {
    'Ti': 'Ti.pbe-spn-kjpaw_psl.1.0.0.UPF',
    "La": "La.paw.z_11.atompaw.wentzcovitch.v1.2.upf",
    "Co": "Co_pbe_v1.2.uspp.F.UPF",
    "O": "O.pbe-n-kjpaw_psl.0.1.UPF",
    "H": "H.pbe-rrkjus_psl.1.0.0.UPF",
    "Sr": "Sr_pbe_v1.uspp.F.UPF"
}

# Input data for QE .pwi files
input_data = {
    'system': {
        'ecutwfc': 60, 
        'ecutrho': 480,
    },
    'control': {
        'calculation': 'scf',
        'restart_mode': 'from_scratch',
        'prefix': 'pwscf',
        'pseudo_dir': '/path/to/pseudopotentials',
        'outdir': './tmp',
        'disk_io': 'low',
        'verbosity': 'high'
    },
    'electrons': {
        'conv_thr': 1e-8,
        'mixing_mode': 'local-TF',
        'mixing_beta': 0.35,
        'diagonalization': 'david',
    }
}


# Test & run

### Test absorbate positions (A site up)

In [34]:
read_material = io.read(material)

# H
# add_adsorbate(read_material, "H", height=(3), position=(1.1179142857142858, 1.1179142857142858))

# O
# add_adsorbate(material, "O", height(3), position=(1.1179142857142858, 1.1179142857142858))

# OH
# add_adsorbate(material, "O", height=3, position=(1.1179142857142858, 1.1179142857142858))
# add_adsorbate(material, "H", height=(3+.97907), position=(1.1179142857142858, 1.1179142857142858))

# OOH
# add_adsorbate(read_material, "O", height=(3), position=(1.1179142857142858, 1.1179142857142858))
# add_adsorbate(read_material, "O", height=(3+1.245956), position=(1.1179142857142858, 1.1179142857142858))
# add_adsorbate(read_material, "H", height=(3+.97907 + 1.245956), position=(1.1179142857142858, 1.1179142857142858))

view(read_material)


<Popen: returncode: None args: ['c:\\Users\\zayah\\OneDrive\\desktop\\BSU-CO...>

In [108]:
import os 
import shutil
from ase import io
from ase.build import add_adsorbate
from ase.calculators.espresso import Espresso
from ase import atoms



# Setup (General)
# Define material structure with either .xyz or .cif file
material = "SrCoO3_A.xyz"

# Define the name of the save files
save_name = "SrCoO3_A_OOH"

# Directory to save the files internally (within the current folder)
save_dir = "SrTiO3_A_files"

# Define aborbate name
options = ["O", "H", "OOH", "OH", "H2O"]
abs_name = options[2]

pseudopotentials = {
    "O": "O.pbe-n-kjpaw_psl.0.1.UPF",
    "H": "H.pbe-rrkjus_psl.0.1.UPF",
    "Sr": "Sr.pbe-spn-rrkjus_psl.1.0.0.UPF",
    'Ti': 'ti_pbe_v1.4.uspp.F.UPF',
}
# Input data for QE .pwi files
input_data = {
    'system': {
        'ecutwfc': 60, 
        'ecutrho': 480,
        'occupations': 'smearing',
        'smearing': 'gaussian',
        'deguass': 0.02, # check previous calcs for this value
    },
    'control': {
        'calculation': 'scf',
        'restart_mode': 'from_scratch',
        'prefix': 'pwscf',
        'pseudo_dir':'SSSP_acc_PBE',
        'outdir': './tmp',
        'verbosity': 'high',
        'tprnfor': True,
    },
    'electrons': {
        'conv_thr': 1e-8,
        'mixing_mode': 'local-TF',
        'mixing_beta': 0.35,
        'diagonalization': 'david',
    }
}

profile = EspressoProfile()

# Read the material
read_material = io.read(material)
# The x and y position are calculated by the equations x = 7.8254 * (i / 7) and y = 7.8254 * (j / 7)
if abs_name == "OOH":
    for i in range(8):
        for j in range(8):
            prev = read_material.copy()

            # Add adsorbates
            add_adsorbate(read_material, "O", height=3, position=(7.8254 * (i / 7), 7.8254 * (j / 7)))
            add_adsorbate(read_material, "O", height=3, position=(7.8254 * (i / 7), 7.8254 * (j / 7)))
            add_adsorbate(read_material, "H", height=3, position=(7.8254 * (i / 7), 7.8254 * (j / 7)))

            # Define directories and file paths
            bin_folder = "0" if i < 4 else "1"
            dir_name = f"{i}-{j}"
            file_name = f"{i}-{j}.xyz"
            file_path = os.path.join(save_dir, bin_folder, dir_name)

            # Create directories if they don't exist
            if not os.path.exists(file_path):
                os.makedirs(file_path)

            io.write(os.path.join(file_path, file_name), read_material)     
            # Create calculator object
            calc = Espresso(
                profile=profile,
                pseudopotentials=pseudopotentials,
                input_data=input_data,
                kpts=(4, 4, 1),
                koffset=(0, 0, 0),
            )

            # Set the calculator and write input
            read_material.calc = calc

            io.espresso.write_espresso_in(os.path.join(file_path, "espresso.pwi"), read_material, input_data=input_data,pseudopotentials=pseudopotentials, kpts=(4, 4, 1), koffset=(0, 0, 0))
            # Restore the material to its previous state
            read_material = prev.copy()


dict_keys(['O', 'H', 'Sr', 'Ti'])


In [38]:

if abs_name == "OH":
    for i in range(8):
        for j in range(8):
            prev = read_material.copy()

            add_adsorbate(read_material, "O", height=3, position=(7.8254 * (i / 7), 7.8254 * (j / 7)))
            add_adsorbate(read_material, "H", height=(3+.97907), position=(7.8254 * (i / 7), 7.8254 * (j / 7)))

            # save file to .xyz then remove the atoms just added
            write(f"{save_name}-{abs_name}-{i}-{j}.xyz", read_material)
            read_material = prev.copy()
elif abs_name == "O":
    for i in range(8):
        for j in range(8):
            prev = read_material.copy()
            add_adsorbate(read_material, "O", height=3, position=(7.8254 * (i / 7), 7.8254 * (j / 7)))

            # save file to .xyz then remove the atoms just added
            write(f"{save_name}-{abs_name}-{i}-{j}.xyz", read_material)
            read_material = prev.copy()
elif abs_name == "H":
    for i in range(8):
        for j in range(8):
            prev = read_material.copy()
            add_adsorbate(read_material, "H", height=(3), position=(7.8254 * (i / 7), 7.8254 * (j / 7)))

            # save file to .xyz then remove the atoms just added
            write(f"{save_name}-{abs_name}-{i}-{j}.xyz", read_material)
            read_material = prev.copy()


SyntaxError: invalid syntax (760938132.py, line 1)

# K-Point Covnergence

In [ ]:
kpoints = range(1,9) 
energies = []
for k in kpoints:
    calc.set(kpts=(k, k, k))
    energies.append(atoms.get_potential_energy())
    

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

plt.plot(kpoints, energies)
plt.xlabel("No. K-Points")
plt.ylabel("E (eV)")
plt.tight_layout()
plt.show()

In [ ]:
calc.set(kpts = (8,8,8))

# Equation of State Fitting

## Automatic

In [ ]:
from ase.eos import EquationOfState, calculate_eos
eos = calculate_eos(atoms, trajectory='GaAs_SJ.traj', npoints=20, eps=0.75)
v, e, B = eos.fit()
eos.plot()
pass

In [ ]:
from ase import units
print(B / units.kJ * 1.0e24, 'GPa')

## Manual

In [ ]:
from ase.io.trajectory import Trajectory
import numpy as np

atoms = read("GaAs.cif")
cell = atoms.get_cell()
atoms.calc = calc

traj = Trajectory('GaAs_murnaghan.traj', 'w')
for x in np.linspace(0.95, 1.05, 5):
    atoms.set_cell(cell * x, scale_atoms=True)
    atoms.get_potential_energy()
    traj.write(atoms)
    

See full list of EOS options at https://databases.fysik.dtu.dk/ase/ase/eos.html

In [ ]:
configs = read("GaAs_murnaghan.traj@0:5")
volumes = [ag.get_volume() for ag in configs]
energies = [ag.get_potential_energy() for ag in configs]
eos = EquationOfState(volumes, energies, eos="murnaghan")
v0, e0, B = eos.fit()
print(B / units.kJ * 1.0e24, 'GPa')
eos.plot()
pass

# Density of States

In [ ]:
from ase.dft.dos import DOS
atoms = read("GaAs.cif")
cell = atoms.get_cell()
atoms.calc = calc
atoms.get_potential_energy()
efermi = calc.get_fermi_level()
dos = DOS(calc, width=0.2)
d = dos.get_dos()
e = dos.get_energies()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(e, d)
plt.vlines(x=efermi, ymin=0, ymax=200, color="k", linestyle=":")
plt.xlabel('energy [eV]')
plt.ylabel('DOS')
plt.show()

In [ ]:
from ase.dft import get_distribution_moment
volume = get_distribution_moment(e,d)
center, width = get_distribution_moment(e,d,(1,2))
print(center, "+/-", width)

# Band Structure

In [ ]:
atoms.get_cell()

In [ ]:
lat = atoms.cell.get_bravais_lattice()
print(lat.description())
lat.plot_bz(show=True)

In [ ]:
kpath = atoms.cell.bandpath()
kpath

In [ ]:
input_data['control'].update(
    {
        'calculation':'bands',
        'restart_mode':'restart',
        'verbosity':'high'
    }
)
calc.set(
    kpts=kpath,
    input_data=input_data
)

calc.calculate(atoms)


In [ ]:
bs = calc.band_structure()
bs.plot(emin=-10, emax=10)